# 字元 (Char) 與字串 (String)

Julia 字元和字串的相關型別及階層關係如下圖。`Any` 是所有型別的最上層父型別。

![](char_string_types.png)

## 1. 字元

### 1.1 字元的宣告

In [1]:
'x'

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [2]:
# answer 為字元變數，使用 typeof 函式可以看到其型號為字元
answer = 'y'
typeof(answer)

Char

In [3]:
# 使用 \u 或 \U 加上 Unicode 編碼，可以顯示相對應的 Unicode 字元
'\u2460'

'①': Unicode U+2460 (category No: Number, other)

In [4]:
# 同樣的，我們可以看到 2460 這個 Unicode 字元的型別為 Char
typeof('①')

Char

### 1.2 字元與數值的轉型

#### 將數值轉型為字元

將 10 進位數值轉型為字元

In [5]:
Char(120)

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

由於使用 Char 轉型並不會判斷字元編碼是否為合法值，這時候可以用 `isvalid()` 函式來判斷。合法的話，會傳回 true 值

In [6]:
println(isvalid(Char, 120))

true


如果編碼值為非法的話，則會傳回 false 值

In [7]:
println(isvalid(Char, 0x110000))

false


#### 將字元轉型為數值

In [8]:
convert(Int64, 'x')

120

#### 判斷字元是否是 ascii 字元

要判斷字元是否是 ascii 字元，可以使用 `isascii()` 函式

In [9]:
println(isascii('x'))
println(isascii('\u2460')) # ① 字元

true
false


### 1.3 字元的運算與比較

字元跟字元之間可用減法，計算兩個字元間編碼的距離。但是加、乘、除法均不合法。

字元本身加減整數值，可以得到該字元往前或往後對應的字元。

In [10]:
'z' - 'a'

25

字元本身加減整數值，可以得到該字元往前或往後對應的字元。

In [11]:
'A' + 25

'Z': ASCII/Unicode U+005a (category Lu: Letter, uppercase)

In [12]:
'x' + 100

'Ü': Unicode U+00dc (category Lu: Letter, uppercase)

字元之間可以使用比較運算子進行比較

In [13]:
'A' < 'a'

true

## 2. 字串

### 2.1 字串的宣告

字串是以成對雙引號或是成對的 3 個雙引號。

In [14]:
x = "Hello Julia"
typeof(x)

String

In [15]:
# 成對的 3 個雙引號
typeof("""Hello Julia""")

String

In [16]:
# 字串中也可以包含 Unicode 字元
y = "\u2200 x \u2203 y"

"∀ x ∃ y"

若是字串中要包含引號的話, 可以用下列兩種方式:
- 成對引號中, 使用\\加引號
- 成對3引號中, 使用引號

In [17]:
println("方式1: 成對引號中, 使用\\加引號")
println("Hello \"Julia\" world\n")
println("方式2: 成對3引號中, 使用引號")
println("""Hello "Julia" world""")

方式1: 成對引號中, 使用\加引號
Hello "Julia" world

方式2: 成對3引號中, 使用引號
Hello "Julia" world


Julia 的字串也支援 C 語言的跳脫序列 (escape sequence)。下面範例是在字串中加入換行 `\n`。

詳細說明可以參考 Wikipedia [C syntax](https://en.wikipedia.org/wiki/C_syntax#Backslash_escapes)

In [18]:
str = "hello\nword"

"hello\nword"

In [19]:
println(str)

hello
word


### 2.2 字串的索引

字串可以透過索引值，取得對應位置的字元或子字串

字串的索引起始值是從 1 開始，在 Julia 語言中預設皆是如此，與其他大多程式語言有所不同

In [20]:
# 取得 x 字串的第 3 個字元
x[3]

'l': ASCII/Unicode U+006c (category Ll: Letter, lowercase)

In [21]:
# 取得 x 字串的第 2 - 最後一個字元的子字串
x[7:end]

"Julia"

#### 字串中有 UTF-8 編碼字元時計算索引位置

下列字串包含 UTF-8 字元。

In [22]:
str = "\u2200x\u2203y" 

"∀x∃y"

In [23]:
str[1]

'∀': Unicode U+2200 (category Sm: Symbol, math)

因為 UTF-8 字元長度不一定相同，所以用索引有可能無法定位到正確的字元，而會產生錯誤。

In [24]:
str[2]

StringIndexError: StringIndexError("∀x∃y", 2)

呼叫 `length()` 函式算出字串的長度，我們可以看到是 4 個字元組成的字串。

但是若以索引數來看的話，索引長度是 8。`firstindex()` 回傳是字串中第一個索引值，`lastindex()` 回傳是字串中最後一個索引值。

In [25]:
println("字元數目：", length(str))
println("索引長度：", lastindex(str) - firstindex(str) + 1)

字元數目：4
索引長度：8


呼叫 `nextind()` 函式，可以找出正確的 index。在下面例子中，先用 `nextind()` 找出所有在字串中每個字元正確的 index，之後再透過 index 印出每個字元。

取得的索引值為存在陣列 (array) 中，陣列我們會在接下來的內容中詳細介紹。

In [26]:
a = []
i = 0

while i < lastindex(str)
    i = nextind(str, i)
    push!(a, i)
end

In [27]:
# 列出原先字串中的每個字元
for j in a
    print(str[j])
end

∀x∃y

#### 字串的插值 (Interpolation)

在應用上，常會遇到需要在字串中插入變數值，與字串結合或輸出，這時候我們可以在字串使用 $，$ 後接續變數名稱或是表達式 (expression)，就可以達到將字串中的變數值或是表達式整合在一起。

In [28]:
greet = "Welcome"
whom = "Julia"

print("$greet $whom")

Welcome Julia

In [29]:
"1 + 2 = $(1 + 2)"

"1 + 2 = 3"

#### 字串常用操作 – 組合

字串與字串或字元之間的組合，可以透過下列幾種方式進行操作：
- `string()` 函式
- `*` 運算子
- Broadcast 函式

In [30]:
string("abc", "123")

"abc123"

In [31]:
"abc" * "123"

"abc123"

In [32]:
broadcast(*, "abc", "123")

"abc123"

若是要結合字串/字元元組 (tuple) 或是陣列 (array)，可以使用 `join()` 函式，也可以使用 String() 建構子。

In [33]:
join(["abc" "123"])

"abc123"

In [34]:
String(['a', 'b', 'c'])

"abc"

#### 字串常用操作 – 比較

字串與字串之間可以用比較運算子進行比較

In [35]:
"abc" < "xyz"

true

但是字串和字元之間不能互相比較大小，只能比較是否相等

In [36]:
# 產生 exception
"abc" < 'a'

MethodError: MethodError: no method matching isless(::String, ::Char)
Closest candidates are:
  isless(!Matched::Char, ::Char) at char.jl:209
  isless(!Matched::Missing, ::Any) at missing.jl:87
  isless(!Matched::AbstractChar, ::AbstractChar) at char.jl:216
  ...

In [37]:
"abc" == 'a'

false

#### 字串常用操作 – 搜尋

要搜尋子字串是否在字串內，可使用 `occursin()` 函式

In [38]:
occursin("world", "hello world")

true

要搜尋字元是否在字串內，可使用 in 或是 ∈

In [39]:
'w' ∈ "hello world"

true

`findfirst()`, `findlast()`, `findprev()`, `findnext()` 函式

In [40]:
# 找第一個
findfirst("o", "hello world")

5:5

In [41]:
# 找最後一個
findlast("o", "hello world")

8:8

若要比較字元的話，不能直接比較，需要用下列語法進行比較。

回傳值與字串互比略為不同，回傳的是絕對的位置。

In [42]:
findlast(isequal('o'), "hello world")

8

In [43]:
# 找指定位置的找下一個符合字元
findnext(isequal('l'), "hello world", 6)

10

In [44]:
# 找指定位置的前一個符合字元
findprev(isequal('o'), "hello world", 5)

5

#### 字串常用操作 – 取代

取代字串中的內容，可以使用 `replace()` 函式，由於字串本身是 immutable，所以使用 `replace()` 函式不會變更原字串。

In [45]:
replace("hello world", "o"=>"p")

"hellp wprld"

搭配使用 count，可以指定要取代的數目。下例僅會取代第一個 o。

In [46]:
replace("hello world", "o"=>"p"; count=1)

"hellp world"

#### 字串常用操作 – 分割

分割字串，例如要分割一個逗號分隔的字串時，可以使用 `split()` 函式，分割後的字串，型別為 SubString。

In [47]:
split("hello, world, John, Wick", ',')

4-element Array{SubString{String},1}:
 "hello" 
 " world"
 " John" 
 " Wick" 

#### 字串常用操作 – 與數值之間的轉型

要將數值轉型為字串，可以使用 `string()` 函式

In [48]:
string(10)

"10"

轉型時，加上 pad 參數可以補足位數

下面例子是110 之前補 7 個 0，產生總共 10 位數長的字串。

In [49]:
string(110, base=10, pad=10)

"0000000110"

要將字串轉型為數值，可使用 `parse()` 函式

In [50]:
parse(Float64, "12.3")

12.3

In [51]:
typeof(ans)

Float64